In [1]:
import pandas as pd
import os
from skimage import io
from skimage import color
from PIL import Image
import numpy as np
import shutil
L2 = lambda x,y: (x**2 + y**2)**0.5

In [2]:
# sciezka do rozpakowanych zdjec
path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) + "/images/"
names = os.listdir(path)

contents = [" ".join(el.replace("-"," ").split(" ")[2:-1]) for el in names]
ids = [el.split("-")[-1].split(".")[0] for el in names]
widths = [Image.open(path + name).size[0] for name in names]
heights = [Image.open(path + name).size[1] for name in names]
means = [np.mean(io.imread(path + name),axis=(0,1)) for name in names]
medians = [np.median(color.rgb2gray(io.imread(path + name))) for name in names]

In [10]:
def brightest_coords(imagepath):
    # image[y][x]
    image = color.rgb2gray(io.imread(imagepath))
    x_max = 0
    y_max = 0
    curr_max = image[0][0]
    for y in range(len(image)):
        for x in range(len(image[1])):
            if image[y][x]==curr_max:
                if L2(x,y)<L2(x_max,y_max):
                    x_max = x
                    y_max = y
            if image[y][x]>curr_max:
                curr_max = image[y][x]
                x_max = x
                y_max = y
    return (x_max,y_max)

coords = [brightest_coords(path + name) for name in names]

igreks = [el[1] for el in coords]
exes = [el[0] for el in coords]

In [11]:
dd = {"Nazwy": names, "Opis": contents, "Id": ids, "Szerokosc": widths, "Wysokosc": heights, "Sredni kolor": means, "Mediana jasnosci": medians, "Pionowy najjasniejszego": igreks, "Poziomy najjasniejszego": exes}
frame = pd.DataFrame(data=dd)
frame.to_csv("images.csv")

#Zadanie Drugie

In [4]:
sorted_by_brightnesses = [el[1] for el in  sorted([[median,name] for name,median in zip(names,medians)])]
folder = os.getcwd() + "/agg-images"
try:
    os.mkdir(folder)
except:
    pass
for i in range(0,len(names)//4):
    bucket = sorted_by_brightnesses[0+4*i:4+4*i]
    numfolder = folder + f"/{i+1}-images"
    try:
        os.mkdir(numfolder)
    except:
        pass
    for photo in bucket:
        shutil.copyfile(path+photo,numfolder + "/" + photo)
# print()
shutil.make_archive(os.getcwd()+"/agg-images","zip",os.getcwd() +"/agg-images")
shutil.rmtree(folder)
